*Load packages and data:*

In [1]:
%run stdPackages.ipynb
read = {'variables': ['Fundamentals', 'Load', 'Generators_Other','TL'], 
        'variable2D': ['Generators_FuelMix','HourlyVariation'],
        'scalars': ['Scalars'],
        'maps': ['Generators_Categories', 'Load_Categories']}
db = dbFromWB(os.path.join(d['data'],'E3.xlsx'), read)
readSets(db)

# The ```mBasicTrade``` model

### **The model**

The class specifies the linear electricity system model specified as:
<a id='simplemodel'>
$$\begin{align} \tag{1}
    \max \text{Welfare} &= \sum_{h,g}\left(u\cdot D_h^g-\sum_{i\in\mathcal{I}_g}mc_{id}\cdot E_{id,h}\right) - \left(\sum_{l,h} c_{l}\cdot x_{l,h}\right)\\ 
               D_h^g &= \sum_{id\in\mathcal{I}_g} E_{id,h}+\sum_{l\in S_g}(1-\text{ll})\cdot x_{l',h}-x_{l,h} \\ 
               D_h^g &\in [0, z_h\cdot \text{Load}_g], \\
               E_{id,h}&\in[0,q_{id,h}] \\
               x_{l,h} &\in[0, q_l].
\end{align}$$

* $u$ is the consumers' marginal willingness to pay for not having the load shedded. 
* $D_h^g$ is the actual served load in hour $h$, area $g$ whereas ($\text{Load}_g\cdot z_h$) is the planned level of demand/load in hour, area $h,g$; 
* $z_h$ is a measure of the hourly variation in demand. If we make sure that $\sum_h z_h =1$ the 'Load' parameter is a measure of 'yearly'/long run demand. 
* ($z_h\cdot \text{Load} - D_h$) measures the shedded load in a given hour. 
* $x_{l,h}$ is the amount of electricity transmitted through line $l=(g,g')$, $c_l$ is the marginal cost of using the line, and $\text{ll}$ is the rate of loss from the transmission line. Thus, $(1-\text{ll})x_{l',h}-x_{l,h}$ is the *net import* of electricity in area $g$ from area $g'$. 
* The boundary constraint $D_h^g\in[0, z_h\cdot \text{Load}_g]$ states that demand cannot be negative and that load shedding cannot be negative.
* The boundary constraint $E_{id,h}\in[0,q_{id,h}]$ states the generation cannot be negative and not exceed a plant-and-hourly specific generation capacity. This generation capacity is generally constant for 'standard' plants, but varies with hours for solar and wind type of plants.
* The boundary constraint $x_{l,h}\in[0,q_l]$ states that transmission can only be positive up to the capacity limit $q_l$. Note that $q_l$ measures the **export capacity**; when this line is fully utilized the received amount of electricity in $g'$ is given by $(1-\text{ll})x_{l'}$.

### 1. **Augmented form LP**

For the previous models (```mBasic```, ```mBasicInt```), we outlined the complete augmented form LP of the model. This is, however, starting to be quite a cumbersome task...

## 2. Initialize and solve model

The model is initialized with the database as input using: 

In [2]:
m = mBasicTrade.mBasicTrade(db)

All models are provided with three main methods: ```self.preSolve```, ```self.initBlocks```, and ```self.postSolve```. The ```preSolve``` and ```postSolve``` methods are used to carry out computations before and after the model is solved, respectively. The ```self.preSolve``` and ```self.postSolve``` are similar to the ones in the [mBasicInt model](M_mBasicInt.ipynb). We focus on the ```self.initBlocks``` here. We note that the following statement carries out all three steps:

In [3]:
m.solve()

Solution status 0: Optimization terminated successfully.


#### 2.1. The vector of cost coefficients $c$:

<img src="snippets/mBasicTrade_snippet.png" width="1000" height="400">

*$c$ component for generation: Marginal costs repeated for all $id,g$ combinations in ```Generation``` variable:*

In [4]:
lpCompiler.broadcast(m.db['mc'], m.globalDomains['Generation'])

id   g   h
id1  g1  1     6.189324
         2     6.189324
         3     6.189324
         4     6.189324
id2  g1  1     9.560114
         2     9.560114
         3     9.560114
         4     9.560114
id3  g1  1    15.402685
         2    15.402685
         3    15.402685
         4    15.402685
id4  g1  1          3.0
         2          3.0
         3          3.0
         4          3.0
id5  g2  1     9.560114
         2     9.560114
         3     9.560114
         4     9.560114
id6  g2  1          3.0
         2          3.0
         3          3.0
         4          3.0
dtype: object

*$c$ component for hourly demand: Minus marginal willingness to pay repeated for all $g,h$ in ```HourlyDemand``` variable:*

In [5]:
m.blocks.get(('c','HourlyDemand'))

g   h
g1  1   -25.0
    2   -25.0
    3   -25.0
    4   -25.0
g2  1   -25.0
    2   -25.0
    3   -25.0
    4   -25.0
Name: 0, dtype: float64

*$c$ component for transmission: Marginal costs (```lineMC```) repeated for all $h$:*

In [6]:
m.blocks.get(('c','Transmission'))

g   g_alias  h
g1  g2       1    0.1
             2    0.1
             3    0.1
             4    0.1
g2  g1       1    0.1
             2    0.1
             3    0.1
             4    0.1
Name: 0, dtype: object

#### 2.2. The vector of upper bounds $u$:

The upper bounds consists of hourly generation, demand, and transmission capacity constraints:

<img src="snippets/mBasicTrade_snippet2.png" width="1200" height="400">

The two methods ```self.hourlyGeneratingCapacity``` and ```self.hourlyLoad``` are specified elsewhere in the class. This is very similar to the restrictions for the model ```mBasicInt```.

#### 2.3. Constants in equality constraints $b_{eq}$:
Consists of zeros for each equilibrium constraint (defined for each $h,g$)

<img src="snippets/mBasicInt_snippet3.png" width="900" height="400">

Note that we use the same trick here as we did with the $c$ coefficient: In ```self.globalDomains``` we have specified that the ```equilibrium``` constraint is defined over the sets ```h_alias``` and ```g_alias2``` (aliases of $h,g$; a copy with a separate identifier). Thus, when we add a parameter with ```None``` as here, we automatically broadcast this as a vector of zeros.

In [7]:
m.blocks.get(('eq','b','equilibrium'))

g_alias2  h_alias
g1        1          0
          2          0
          3          0
          4          0
g2        1          0
          2          0
          3          0
          4          0
Name: 0, dtype: int64

#### 2.4. Coefficients in equality constraints $A_{eq}$:

The coefficients in the equilibrium constraint are starting to become quite a block. The equilibrium constrained is defined over the sets ```g_alias2, h_alias``` aliases for $g,h$. All three variables (```Generation```, ```HourlyDemand```, ```Transmission```) enters the equilibrium - and they are all defined over $h,g$ as well. So, let us take them one at a time.

<img src="snippets/mBasicTrade_snippet3.png" width="1500" height="600">

**Generation:** 

We start by creating a vector of $1$ defined over the domains of ```Generation``` variable. Next, we use the ```appIndexWithCopySeries``` method which adds new index levels that are copies of existing levels. So, in this case, the sets $(g,h)$ are repeated and given the new aliased names:

In [8]:
generation = mBasicTrade.appIndexWithCopySeries(pd.Series(1, index = m.globalDomains['Generation']), ['g','h'], ['g_alias2','h_alias']) # Note that h = h_alias and g=g_alias2
generation

id   g   h  g_alias2  h_alias
id1  g1  1  g1        1          1
         2  g1        2          1
         3  g1        3          1
         4  g1        4          1
id2  g1  1  g1        1          1
         2  g1        2          1
         3  g1        3          1
         4  g1        4          1
id3  g1  1  g1        1          1
         2  g1        2          1
         3  g1        3          1
         4  g1        4          1
id4  g1  1  g1        1          1
         2  g1        2          1
         3  g1        3          1
         4  g1        4          1
id5  g2  1  g2        1          1
         2  g2        2          1
         3  g2        3          1
         4  g2        4          1
id6  g2  1  g2        1          1
         2  g2        2          1
         3  g2        3          1
         4  g2        4          1
dtype: int64

The point of this is that when we construct the matrix $A$, we can use the ```unstack``` method on the indices on the constraint, and we automatically have the right matrix. So, the ```lpCompiler``` class will essentially do something like this:

In [9]:
generation.unstack(m.globalDomains['equilibrium'].names).fillna(0)

g_alias2   g1                  g2               
h_alias     1    2    3    4    1    2    3    4
id  g  h                                        
id1 g1 1  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
       2  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
       3  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
       4  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
id2 g1 1  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
       2  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
       3  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
       4  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
id3 g1 1  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
       2  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
       3  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
       4  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
id4 g1 1  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
       2  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
       3  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
       4  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
id5 g2 1  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
       2  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
       3  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
       4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
id6 g2 1  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
       2  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
       3  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
       4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

Note that this has $1$ in the diagonal where ```h=h_alias``` and ```g=g_alias2``` and zeros elsewhere.

**HourlyDemand:** 

We start by creating a vector of $-1$ defined over the domains of ```HourlyDemand``` variable. Next, we use the ```appIndexWithCopySeries``` method which adds new index levels that are copies of existing levels. So, in this case, the sets $(g,h)$ are repeated and given the new aliased names:

In [10]:
demand = mBasicTrade.appIndexWithCopySeries(pd.Series(-1, index = m.globalDomains['HourlyDemand']), ['g','h'], ['g_alias2','h_alias']) # Note that h = h_alias and g=g_alias2
demand

g   h  g_alias2  h_alias
g1  1  g1        1         -1
    2  g1        2         -1
    3  g1        3         -1
    4  g1        4         -1
g2  1  g2        1         -1
    2  g2        2         -1
    3  g2        3         -1
    4  g2        4         -1
dtype: int64

Similarly, as with the ```Generation``` variable, we use the ```unstack``` method when we create the $A$ matrix in the linear program. So, the ```lpCompiler``` class will essentially do something like this:

In [11]:
demand.unstack(m.globalDomains['equilibrium'].names).fillna(0)

g_alias2   g1                  g2               
h_alias     1    2    3    4    1    2    3    4
g  h                                            
g1 1     -1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
   2      0.0 -1.0  0.0  0.0  0.0  0.0  0.0  0.0
   3      0.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0
   4      0.0  0.0  0.0 -1.0  0.0  0.0  0.0  0.0
g2 1      0.0  0.0  0.0  0.0 -1.0  0.0  0.0  0.0
   2      0.0  0.0  0.0  0.0  0.0 -1.0  0.0  0.0
   3      0.0  0.0  0.0  0.0  0.0  0.0 -1.0  0.0
   4      0.0  0.0  0.0  0.0  0.0  0.0  0.0 -1.0

**Transmission:** 

This is by far the most tricky part to add, in particular because we have to subtract $x_{l,h}$ and add $(1-\text{ll})x_{l',h}$ where $l' = (g',g)$. So, let us start by subtract $x_{l,h}$: This is essentially done as with the other variables:

In [12]:
transmission_1 = mBasicTrade.appIndexWithCopySeries(pd.Series(-1, index = m.globalDomains['Transmission']), ['g','h'], ['g_alias2','h_alias']) # Note that h = h_alias and g=g_alias2
transmission_1

g   g_alias  h  g_alias2  h_alias
g1  g2       1  g1        1         -1
             2  g1        2         -1
             3  g1        3         -1
             4  g1        4         -1
g2  g1       1  g2        1         -1
             2  g2        2         -1
             3  g2        3         -1
             4  g2        4         -1
dtype: int64

When this is unstacked (like we did earlier), this adds a -1 in the diagonal:

In [13]:
transmission_1.unstack(m.globalDomains['equilibrium'].names).fillna(0)

g_alias2       g1                  g2               
h_alias         1    2    3    4    1    2    3    4
g  g_alias h                                        
g1 g2      1 -1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
           2  0.0 -1.0  0.0  0.0  0.0  0.0  0.0  0.0
           3  0.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0
           4  0.0  0.0  0.0 -1.0  0.0  0.0  0.0  0.0
g2 g1      1  0.0  0.0  0.0  0.0 -1.0  0.0  0.0  0.0
           2  0.0  0.0  0.0  0.0  0.0 -1.0  0.0  0.0
           3  0.0  0.0  0.0  0.0  0.0  0.0 -1.0  0.0
           4  0.0  0.0  0.0  0.0  0.0  0.0  0.0 -1.0

Note that if there are more than two areas, this would create $-1$ for all ```g_alias``` as long as ```g = g_alias2```.

The second part, $(1-\text{ll})\cdot x_{l',h}$, is then added. We do this by repeating the ```g_alias``` index instead of the ```g``` index:

In [14]:
transmission_2 = mBasicTrade.appIndexWithCopySeries(pd.Series(1-m.db['lineLoss'], index = m.globalDomains['Transmission']), ['g_alias','h'], ['g_alias2','h_alias']) # Note that h = h_alias and g_alias=g_alias2
transmission_2

g   g_alias  h  g_alias2  h_alias
g1  g2       1  g2        1          0.95
             2  g2        2          0.95
             3  g2        3          0.95
             4  g2        4          0.95
g2  g1       1  g1        1          0.95
             2  g1        2          0.95
             3  g1        3          0.95
             4  g1        4          0.95
dtype: float64

When this is unstacked this adds $1-\text{ll}$ whenever ```g_alias = g_alias2```:

In [15]:
transmission_2.unstack(m.globalDomains['equilibrium'].names).fillna(0)

g_alias2        g2                      g1                  
h_alias          1     2     3     4     1     2     3     4
g  g_alias h                                                
g1 g2      1  0.95  0.00  0.00  0.00  0.00  0.00  0.00  0.00
           2  0.00  0.95  0.00  0.00  0.00  0.00  0.00  0.00
           3  0.00  0.00  0.95  0.00  0.00  0.00  0.00  0.00
           4  0.00  0.00  0.00  0.95  0.00  0.00  0.00  0.00
g2 g1      1  0.00  0.00  0.00  0.00  0.95  0.00  0.00  0.00
           2  0.00  0.00  0.00  0.00  0.00  0.95  0.00  0.00
           3  0.00  0.00  0.00  0.00  0.00  0.00  0.95  0.00
           4  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.95

When we add the two together, we get the pattern where a component $x_{g,g',h}$ will enter with $-1$ in the equilibrium constraint for $g$ and $1-\text{ll}$ in the constraint for $g'$:

In [16]:
m.blocks.get(('eq','A','equilibrium','Transmission'))

g   g_alias  h  g_alias2  h_alias
g1  g2       1  g1        1         -1.00
                g2        1          0.95
             2  g1        2         -1.00
                g2        2          0.95
             3  g1        3         -1.00
                g2        3          0.95
             4  g1        4         -1.00
                g2        4          0.95
g2  g1       1  g1        1          0.95
                g2        1         -1.00
             2  g1        2          0.95
                g2        2         -1.00
             3  g1        3          0.95
                g2        3         -1.00
             4  g1        4          0.95
                g2        4         -1.00
dtype: float64

**The final matrix, $A$:**

When we take all the components together, you can see why we use the ```lpCompiler``` to set up the model. The final block that is parsed as the matrix $A$ is:

In [17]:
m.blocks.lp_A_eq

array([[ 1.  ,  1.  ,  1.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -1.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  , -1.  ,  0.  ,  0.  ,  0.  ,
         0.95,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  1.  ,  1.  ,  1.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -1.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -1.  ,  0.  ,  0.  ,
         0.  ,  0.95,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,
         1.  ,  1.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -1.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -1.  ,  0.  ,
         0.  ,  0.  ,  0.95,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  